In [28]:
import pickle
import numpy as np
import pandas as pd

In [29]:
customer_data = pd.read_pickle("../data/customer_data.pkl")

In [30]:
customer_data.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

# Generate Rating Dataframe

In [31]:
# Use the customer's holdings as their ratings 

ratingList = []
for customer in customer_data.keys():
    tmp_data = customer_data[customer]
    for stock,rating in zip(tmp_data['stocks'],tmp_data['positions']):
        ratingList.append([customer,stock,rating])

In [32]:
df_rating = pd.DataFrame(ratingList,columns = ['customerID','stockID','rating'])

In [33]:
df_rating.head()

,customerID,stockID,rating
0,0,BLL,0.151362
1,0,TRV,0.383889
2,0,REG,0.241205
3,0,MRO,0.223544
4,1,DLTR,0.327430


In [34]:
df_rating.stockID.unique()

array(['BLL', 'TRV', 'REG', 'MRO', 'DLTR', 'GM', 'IFF', 'GWW', 'TIF',
       'RJF', 'FFIV', 'MRK', 'AVB', 'UAL', 'GLW', 'CINF', 'UNM', 'NI',
       'KO', 'MDLZ', 'HCA', 'DG', 'RHT', 'MET', 'BDX', 'UNP', 'COF',
       'NKE', 'NAVI', 'ABT', 'APA', 'PWR', 'ACN', 'HRL', 'XL', 'SO',
       'DXC', 'TSS', 'NCLH', 'GOOG', 'LRCX', 'ADS', 'JPM', 'AMGN', 'RF',
       'BK', 'PRU', 'SYMC', 'CHD', 'CL', 'AAPL', 'ROK', 'PYPL', 'NOC',
       'VMC', 'ILMN', 'FCX', 'XRX', 'HLT', 'MCD', 'CBS', 'COST', 'ZTS',
       'SWKS', 'BBY', 'F', 'WFC', 'VRSK', 'DHI', 'MO', 'DTE', 'PNC',
       'EQR', 'COTY', 'HPE', 'HSY', 'BHF', 'PLD', 'SBUX', 'HON', 'ICE',
       'FIS', 'LLY', 'AON', 'WLTW', 'PX', 'BHGE', 'DISH', 'MPC', 'HOLX',
       'CAT', 'COG', 'IDXX', 'ALLE', 'ORLY', 'SCG', 'NEM', 'GPC', 'TGT',
       'FTV', 'MHK', 'CPB', 'DIS', 'DE', 'KLAC', 'INCY', 'AMG', 'FLS',
       'NTRS', 'LNC', 'AGN', 'NFX', 'AME', 'ALL', 'VFC', 'MDT', 'SPG',
       'UTX', 'CBOE', 'BSX', 'AET', 'QCOM', 'RL', 'KMB', 'ANDV', 'AAP',
    

In [35]:
# set a dictionary, stock Name = id
stock2id = {}
for id, stock in enumerate(df_rating.stockID.unique()):
    stock2id[stock]=id

In [36]:
df_rating.stockID = df_rating.stockID.apply(lambda x :stock2id[x])

In [37]:
df_rating.head()

,customerID,stockID,rating
0,0,0,0.151362
1,0,1,0.383889
2,0,2,0.241205
3,0,3,0.223544
4,1,4,0.327430


# ALS with Spark

In [38]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel
from pyspark.sql.types import DoubleType, StringType

In [39]:
# Create Spark Context
sc = pyspark.SparkContext.getOrCreate()

sqlContext = SQLContext(sc)

In [40]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [41]:
# after crate a framework, then create Spark Dataframe
df_spark = sqlContext.createDataFrame(df_rating)
df_spark.printSchema()

root
 |-- customerID: long (nullable = true)
 |-- stockID: long (nullable = true)
 |-- rating: double (nullable = true)



In [42]:
df_spark.head(5)

[Row(customerID=0, stockID=0, rating=0.15136185575205705),
 Row(customerID=0, stockID=1, rating=0.3838889682951716),
 Row(customerID=0, stockID=2, rating=0.24120542930578345),
 Row(customerID=0, stockID=3, rating=0.223543746646988),
 Row(customerID=1, stockID=4, rating=0.32742969807467753)]

In [43]:
# Train ALS Model
rank = 5 
# hidden factors/latent factors : what are the factors are you using to predict?
numIterations = 10 
# iterations is the number of iterations of ALS to run.
model = ALS.train(df_spark,rank,numIterations)

In [44]:
display('features for stock one',model.productFeatures().first()) # 只抽取第一个
display('features for user one', model.userFeatures().first()) 

'features for stock one'

(0,
 array('d', [-0.188982293009758, 0.3519147038459778, 0.08194414526224136, 0.09334617108106613, 0.1301187425851822]))

'features for user one'

(0,
 array('d', [-0.30294114351272583, 0.3325137794017792, 0.0035029451828449965, -0.3078697621822357, -0.0015810858458280563]))

In [50]:
# For Product X, find N users to sell to
stockForUser = model.recommendUsers(222,10)
print("Top 10 stock for user 222:\n")        
display(stockForUser)

Top 10 stock for user 222:



[Rating(user=876, product=222, rating=0.3788932747068795),
 Rating(user=500, product=222, rating=0.35948233786024647),
 Rating(user=983, product=222, rating=0.35211209512076436),
 Rating(user=91, product=222, rating=0.3508894471930925),
 Rating(user=997, product=222, rating=0.3428203869879831),
 Rating(user=299, product=222, rating=0.3389099909540052),
 Rating(user=650, product=222, rating=0.33711159784335826),
 Rating(user=600, product=222, rating=0.33469195062684687),
 Rating(user=405, product=222, rating=0.3304485006032104),
 Rating(user=547, product=222, rating=0.3270663371214392)]

In [51]:
# For user y find N products to promote
userForStock = model.recommendProducts(100,10)
print('Top 10 Users to recommend for stock 100')
display(userForStock)

Top 10 Users to recommend for stock 100


[Rating(user=100, product=1, rating=0.3115441025483345),
 Rating(user=100, product=290, rating=0.2975310150930966),
 Rating(user=100, product=334, rating=0.29093983951161234),
 Rating(user=100, product=250, rating=0.2791913009619995),
 Rating(user=100, product=411, rating=0.27638483101740496),
 Rating(user=100, product=77, rating=0.2706919906242813),
 Rating(user=100, product=262, rating=0.26513653445528973),
 Rating(user=100, product=450, rating=0.2617874391700407),
 Rating(user=100, product=218, rating=0.2596260259903587),
 Rating(user=100, product=99, rating=0.2583562606762242)]

In [53]:
# Predict Ratings
ratingPredict = model.predict(222,100)
print("Prediction of rate user 222 to stock 100")
display(ratingPredict)

Prediction of rate user 222 to stock 100


0.17496613682958373

In [48]:
stockFeatures = model.productFeatures()
stockFeatures.collect()

[(0,
  array('d', [-0.188982293009758, 0.3519147038459778, 0.08194414526224136, 0.09334617108106613, 0.1301187425851822])),
 (8,
  array('d', [-0.2560451030731201, 0.2716205418109894, -0.04638230428099632, 0.0769963264465332, 0.3613567352294922])),
 (16,
  array('d', [0.1471559852361679, -0.03794363513588905, 0.3405923843383789, -0.2792068123817444, 0.030147328972816467])),
 (24,
  array('d', [0.1579970121383667, 0.12357600778341293, 0.4649675786495209, 0.21647539734840393, 0.13186022639274597])),
 (32,
  array('d', [-0.029063217341899872, 0.4480007588863373, -0.1919439285993576, -0.20355160534381866, 0.3365710377693176])),
 (40,
  array('d', [-0.5721644759178162, 0.15909983217716217, 0.12625768780708313, -0.08707597851753235, 0.044292740523815155])),
 (48,
  array('d', [0.5378949046134949, -0.2945292294025421, 0.05024736002087593, -0.26756539940834045, 0.13672339916229248])),
 (56,
  array('d', [-0.0016666975570842624, 0.042418595403432846, -0.031269073486328125, -0.5080047249794006, 